In [1]:
import basic_entities as entities
from glob import glob
import os
import scipy.io
import os.path

In [40]:
def load_video(video_path):
    video = entities.BasicVideo(video_path)
    for frame_path in glob('{}/*.png'.format(video_path)):
        
        video.add_frame(entities.BasicFrame(frame_path))
        
    frames = video.get_frames()
    pairs = video.get_frame_pairs()
    
    gt_path = os.path.splitext(video_path.replace('images', 'gt'))[0]
    person_mat_path = '{}_people_full.mat'.format(gt_path)
    
    if not os.path.isfile(person_mat_path):
        return video
        
    mat = scipy.io.loadmat(person_mat_path)
    
    # Add persons to frames
    for idx, person in enumerate(mat['people'][0]):
        point_list = person[0][0][0]
        for i, coordinates in enumerate(point_list):
            frames[int(coordinates[2])-1].add_point(int(coordinates[0]), int(coordinates[1]))
            
            # If two consequative frames have the same person add the tracked person in the pair
            if i+1 < len(point_list) and int(point_list[i+1][2]) == int(coordinates[2]) + 1:
                frame1_id = len(frames[int(coordinates[2])-1].get_centers())-1
                frame2_id = len(frames[int(coordinates[2])].get_centers())-1
                
                pairs[int(coordinates[2])-1].add_point_pair(frame1_id, frame2_id)
    
    video.labeled = True
    return video

In [42]:
videos = glob('../data/ucsdpeds/images/vidf/*')
for video in videos:
    load_video(video)